In [23]:
%pip install torch torchvision
%pip install trimesh numpy scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import json
import trimesh
import os
import random
import shutil

ModuleNotFoundError: No module named 'sklearn'

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Read data

In [28]:
data = pd.read_csv('/home/aalab/Desktop/tripoFT/metadata.csv')

In [29]:
print("The first few rows of the DataFrame:")
print(data.head())

print("\nData info:")
print(data.info())

print("\nData columns:")
print(data.columns)

print("\nData described:")
print(data.describe())

The first few rows of the DataFrame:
                                 filename      size  \
0  cadbury_dairy_milk_chocolate_piece.glb    152796   
1          bonnie_melted_chocolate_ar.glb   8285860   
2              chocobar_ice_cream (1).glb   4499300   
3                           ice_cream.glb  12444648   
4                   chocolate_truffle.glb   2084100   

                                         object_path  
0  /home/aalab/Desktop/tripoFT/Dataset/cadbury_da...  
1  /home/aalab/Desktop/tripoFT/Dataset/bonnie_mel...  
2  /home/aalab/Desktop/tripoFT/Dataset/chocobar_i...  
3  /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb  
4  /home/aalab/Desktop/tripoFT/Dataset/chocolate_...  

Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     145 non-null    object
 1   size         145 non-null    int64 
 2   object_path  145 

Clean Data

In [16]:
data.drop_duplicates(inplace=True)

# Drop rows with missing values in specific columns
columns_with_missing_values = ['filename', 'size', 'object_path']
data.dropna(subset=columns_with_missing_values, inplace=True)

# Convert columns to appropriate data types
data['filename'] = data['filename'].astype(str)
data['size'] = data['size'].astype(str)
data['object_path'] = data['object_path'].astype(str)

# Print cleaned DataFrame
print(data)

                                   filename      size  \
0    cadbury_dairy_milk_chocolate_piece.glb    152796   
1            bonnie_melted_chocolate_ar.glb   8285860   
2                chocobar_ice_cream (1).glb   4499300   
3                             ice_cream.glb  12444648   
4                     chocolate_truffle.glb   2084100   
..                                      ...       ...   
140                     strawberry_cake.glb  25159596   
141                jiggly_bunny_pudding.glb   1026416   
142            kitchen_sink_fiesta_5149.glb  26103980   
143   easter_egg_2024_marbled_chocolate.glb   3743544   
144              paleta_payaso_lollypop.glb    439920   

                                           object_path  
0    /home/aalab/Desktop/tripoFT/Dataset/cadbury_da...  
1    /home/aalab/Desktop/tripoFT/Dataset/bonnie_mel...  
2    /home/aalab/Desktop/tripoFT/Dataset/chocobar_i...  
3    /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb  
4    /home/aalab/Desktop/tripo

Convert 3D Objects into a Usable Format that our model can work with, such as voxel grids. (This is in shape vertex and faces)

In [8]:
import os
import numpy as np
import trimesh

def voxelize_mesh(vertices, faces, grid_size=32):
    """Convert mesh to voxel grid."""
    # Create an empty voxel grid
    voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.float32)

    # Normalize the vertices to fit in the grid
    vertices -= vertices.min(axis=0)
    vertices /= vertices.max(axis=0)
    vertices *= (grid_size - 1)

    # Rasterize the faces into the voxel grid
    for face in faces:
        verts = vertices[face].astype(int)
        x, y, z = zip(*verts)
        voxel_grid[x, y, z] = 1.0  # Mark the presence of vertices in the grid

    return voxel_grid

def convert_glb_to_tensors(input_folder, output_folder, grid_size=32):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop over all .glb files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.glb'):
            # Load the .glb file using trimesh
            file_path = os.path.join(input_folder, file_name)
            mesh = trimesh.load(file_path, force='mesh')

            # Extract vertices and faces
            vertices = mesh.vertices
            faces = mesh.faces

            # Convert mesh to voxel grid
            voxel_grid = voxelize_mesh(vertices, faces, grid_size=grid_size)

            # Add batch and channel dimensions
            voxel_grid = voxel_grid[np.newaxis, np.newaxis, :, :, :]

            # Save the tensor to a .npz file
            tensor_file_name = file_name.replace('.glb', '.npz')
            tensor_file_path = os.path.join(output_folder, tensor_file_name)
            np.savez(tensor_file_path, voxel_grid=voxel_grid)

            print(f'Successfully processed and saved: {tensor_file_name}')

input_folder = '/home/aalab/Desktop/tripoFT/Dataset'
output_folder = '/home/aalab/Desktop/tripoFT/tensors'
convert_glb_to_tensors(input_folder, output_folder)


Successfully processed and saved: cadbury_dairy_milk_chocolate_piece.npz
Successfully processed and saved: bonnie_melted_chocolate_ar.npz
Successfully processed and saved: chocobar_ice_cream (1).npz


KeyboardInterrupt: 

converting to tensor with shape (Batch size, Channel, Depth, Height, Width)

In [32]:
import os
import numpy as np
import trimesh

def voxelize_mesh(mesh, resolution=32):
    """
    Convert a mesh to a voxel grid of the specified resolution.
    """
    # Compute the voxel size
    voxel_size = (mesh.bounds[1] - mesh.bounds[0]).max() / resolution
    # Voxelize the mesh
    voxelized = mesh.voxelized(voxel_size)
    # Convert the voxelized mesh to a dense voxel grid
    filled = voxelized.matrix
    return filled

def convert_glb_to_tensors(input_folder, output_folder, resolution=32):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop over all .glb files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.glb'):
            # Load the .glb file using trimesh
            file_path = os.path.join(input_folder, file_name)
            mesh = trimesh.load(file_path, force='mesh')

            # Voxelize the mesh
            voxel_grid = voxelize_mesh(mesh, resolution)

            # Normalize the voxel grid to match [channels, depth, height, width]
            voxel_grid = voxel_grid[np.newaxis, :, :, :]  # Add channel dimension
            # No need to add an additional batch dimension

            # Save the tensor to a .npz file
            tensor_file_name = file_name.replace('.glb', '.npz')
            tensor_file_path = os.path.join(output_folder, tensor_file_name)
            np.savez(tensor_file_path, voxel_grid=voxel_grid)

            print(f'Successfully processed and saved: {tensor_file_name}')

input_folder = '/home/aalab/Desktop/tripoFT/Dataset'
output_folder = '/home/aalab/Desktop/tripoFT/tensors'
convert_glb_to_tensors(input_folder, output_folder, resolution=32)


Successfully processed and saved: cadbury_dairy_milk_chocolate_piece.npz
Successfully processed and saved: bonnie_melted_chocolate_ar.npz
Successfully processed and saved: chocobar_ice_cream (1).npz
Successfully processed and saved: ice_cream.npz
Successfully processed and saved: chocolate_truffle.npz
Successfully processed and saved: oreo.npz
Successfully processed and saved: heart_cake.npz
Successfully processed and saved: candy_vending_machine.npz
Successfully processed and saved: gummy_bear_bull.npz
Successfully processed and saved: valentines_heart.npz
Successfully processed and saved: beardpapas_smores_puff__scan.npz
Successfully processed and saved: doughnut_pack.npz
Successfully processed and saved: cc0_-_candies.npz
Successfully processed and saved: biscuit.npz
Successfully processed and saved: hot_fudge_warmer.npz
Successfully processed and saved: chocolate_milkshake.npz
Successfully processed and saved: vegan_icecream.npz
Successfully processed and saved: power_shake_chocola

In [33]:
import numpy as np
import os

def check_npz_shapes(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.npz'):
            file_path = os.path.join(folder_path, file_name)
            data = np.load(file_path)
            voxel_grid = data['voxel_grid']
            print(f'{file_name}: {voxel_grid.shape}')

folder_path = '/home/aalab/Desktop/tripoFT/tensors'
check_npz_shapes(folder_path)


cup_cake.npz: (1, 25, 33, 25)
brownie_square.npz: (1, 33, 15, 33)
ice_cream (1).npz: (1, 5, 33, 13)
schokokuchen_lina_2024.npz: (1, 32, 15, 33)
hot_chocolate_with_whipped_cream.npz: (1, 15, 33, 15)
doughnut_pack.npz: (1, 33, 8, 33)
simple_chocolate_bar.npz: (1, 33, 2, 15)
hershey_kiss.npz: (1, 29, 33, 21)
chocobar_ice_cream.npz: (1, 33, 8, 27)
strawberry_cake_low_poly.npz: (1, 31, 16, 33)
paper_cup_perfect.npz: (1, 25, 33, 25)
chocolate_buttercream_cupcake.npz: (1, 33, 25, 32)
krispy_kreme_twix_bar_doughnut.npz: (1, 17, 12, 33)
bonnie_melted_chocolate_ar.npz: (1, 33, 33, 9)
chocolate_cake.npz: (1, 15, 21, 33)
lindt_gold_chocolate_bunny.npz: (1, 19, 33, 27)
the_meadows_arctic_swirl_frozen_custard_dessert.npz: (1, 33, 19, 33)
paleta_payaso.npz: (1, 14, 33, 9)
blue_mm_without_eyes.npz: (1, 33, 24, 11)
fluid_simulation.npz: (1, 33, 17, 33)
cc0_-_candies.npz: (1, 26, 7, 33)
pancakes_low_poly_model.npz: (1, 33, 19, 33)
long_donut.npz: (1, 17, 10, 33)
easter_egg_2024_marbled_chocolate.npz: (1

Data Augmentation

In [36]:
import os
import numpy as np

def rotate_voxel_grid(voxel_grid, axis, angle):
    """
    Rotate the voxel grid along a specified axis by the given angle (in degrees).
    """
    from scipy.ndimage import rotate
    rotated = rotate(voxel_grid, angle, axes=axis, reshape=False, mode='nearest')
    return rotated

def flip_voxel_grid(voxel_grid, axis):
    """
    Flip the voxel grid along a specified axis.
    """
    flipped = np.flip(voxel_grid, axis=axis)
    return flipped

def add_noise_to_voxel_grid(voxel_grid, noise_level=0.01):
    """
    Add Gaussian noise to the voxel grid.
    """
    noise = np.random.normal(0, noise_level, voxel_grid.shape)
    noisy_voxel_grid = voxel_grid + noise
    return np.clip(noisy_voxel_grid, 0, 1)  # Ensure the values are within [0, 1]

def augment_voxel_grid(voxel_grid):
    """
    Apply various augmentations to the voxel grid and return a list of augmented grids.
    """
    augmentations = []

    # Original grid
    augmentations.append(voxel_grid)

    # Rotations
    for axis in [(1, 2), (1, 3), (2, 3)]:
        for angle in [90, 180, 270]:
            augmentations.append(rotate_voxel_grid(voxel_grid, axis, angle))

    # Flips
    for axis in [1, 2, 3]:
        augmentations.append(flip_voxel_grid(voxel_grid, axis))

    # Adding noise
    augmentations.append(add_noise_to_voxel_grid(voxel_grid, noise_level=0.01))

    return augmentations

def augment_tensor_files(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop over all .npz files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npz'):
            # Load the .npz file
            file_path = os.path.join(input_folder, file_name)
            data = np.load(file_path)
            voxel_grid = data['voxel_grid']

            # Apply augmentations
            augmented_voxel_grids = augment_voxel_grid(voxel_grid)

            # Save augmented voxel grids to new .npz files
            base_name = file_name.replace('.npz', '')
            for i, aug_voxel_grid in enumerate(augmented_voxel_grids):
                aug_file_name = f'{base_name}_aug_{i}.npz'
                aug_file_path = os.path.join(output_folder, aug_file_name)
                np.savez(aug_file_path, voxel_grid=aug_voxel_grid)

                print(f'Successfully augmented and saved: {aug_file_name}')

# Example usage
input_folder = '/home/aalab/Desktop/tripoFT/tensors'
output_folder = '/home/aalab/Desktop/tripoFT/augmented_tensors'
augment_tensor_files(input_folder, output_folder)


Successfully augmented and saved: cup_cake_aug_0.npz
Successfully augmented and saved: cup_cake_aug_1.npz
Successfully augmented and saved: cup_cake_aug_2.npz
Successfully augmented and saved: cup_cake_aug_3.npz
Successfully augmented and saved: cup_cake_aug_4.npz
Successfully augmented and saved: cup_cake_aug_5.npz
Successfully augmented and saved: cup_cake_aug_6.npz
Successfully augmented and saved: cup_cake_aug_7.npz
Successfully augmented and saved: cup_cake_aug_8.npz
Successfully augmented and saved: cup_cake_aug_9.npz
Successfully augmented and saved: cup_cake_aug_10.npz
Successfully augmented and saved: cup_cake_aug_11.npz
Successfully augmented and saved: cup_cake_aug_12.npz
Successfully augmented and saved: cup_cake_aug_13.npz
Successfully augmented and saved: brownie_square_aug_0.npz
Successfully augmented and saved: brownie_square_aug_1.npz
Successfully augmented and saved: brownie_square_aug_2.npz
Successfully augmented and saved: brownie_square_aug_3.npz
Successfully augme

Split the data

In [37]:
import random
import shutil

tensor_directory = '/home/aalab/Desktop/tripoFT/augmented_tensors'

# Directory to save the splits
train_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# # Create directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Set the seed for reproducibility
random.seed(42)

# List of voxel tensor files
tensor_files = os.listdir(tensor_directory)
random.shuffle(tensor_files)

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate the number of files for each split
num_files = len(tensor_files)
num_train = int(train_ratio * num_files)
num_val = int(val_ratio * num_files)

# Split the files
train_files = tensor_files[:num_train]
val_files = tensor_files[num_train:num_train + num_val]
test_files = tensor_files[num_train + num_val:]

# Move files to respective directories
for file in train_files:
    shutil.copyfile(os.path.join(tensor_directory, file), os.path.join(train_dir, file))

for file in val_files:
    shutil.copyfile(os.path.join(tensor_directory, file), os.path.join(val_dir, file))

for file in test_files:
    shutil.copyfile(os.path.join(tensor_directory, file), os.path.join(test_dir, file))

print("Data split into train, validation, and test sets.")
print(num_files, num_train, num_val)

Data split into train, validation, and test sets.
2030 1421 304


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [33]:
import numpy as np
import torch

# Load the .npz file
data = np.load('/home/aalab/Desktop/tripoFT/tensors/a_little_bite_of_france..npz')

# Iterate over the loaded arrays
for key in data:
    # Convert the NumPy array to a PyTorch tensor
    tensor = torch.tensor(data[key])
    # Check the device of the tensor
    print(f"Device of '{key}': {tensor.device}")


Device of 'vertices': cpu
Device of 'faces': cpu


Load the Data

Get the TripoSR Model directory

Training Loop